<a href="https://colab.research.google.com/github/hoo0681/raspi4withTF/blob/master/5_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
 %tensorflow_version 2.x

import tensorflow as tf

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
print("Module Loaded.")
print("TensorFlow Version :{}".format(tf.__version__))
print("NumPy Version :{}".format(np.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

# set auto tune
AUTOTUNE = tf.data.experimental.AUTOTUNE

TensorFlow 2.x selected.
Module Loaded.
TensorFlow Version :2.1.0-rc1
NumPy Version :1.17.5
Matplotlib Version :3.1.2


In [0]:
# 지정 폴더 아래에 있는 모든 *.jpg 파일의 수
#  및 폴더명 목록을 리턴  
def check_dir(d_path):
    img_count = len(list(d_path.glob('*/*.jpg')))
    c_name = np.array([item.name for item in d_path.glob('*') if item.name != "LICENSE.txt"])
    return img_count, c_name

In [0]:
!unzip /content/drive/My\ Drive/tf2/z_data/asirra_fixed.zip

In [0]:
import pathlib
train_data_dir_cwd=pathlib.Path('/content/asirra/train')
test_data_dir_cwd=pathlib.Path('/content/asirra/test')

In [17]:
image_count_cwd, CLASS_NAMES_cwd = check_dir(train_data_dir_cwd)
test_image_count_cwd, CLASS_NAMES_cwd = check_dir(test_data_dir_cwd)
print('image_count: {}\nclasses: {}'.format(image_count_cwd, CLASS_NAMES_cwd))
print('image_count: {}\nclasses: {}'.format(test_image_count_cwd, CLASS_NAMES_cwd))

image_count: 12500
classes: ['dog' 'cat']
image_count: 12500
classes: ['dog' 'cat']


# 변형X 데이터

In [47]:
datagen_2=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
generator_2=datagen_2.flow_from_directory(
    directory=str(train_data_dir_cwd),
    batch_size=12500,
    shuffle=False,
    target_size=(150, 150)
)
generator_2_test=datagen_2.flow_from_directory(
    directory=str(test_data_dir_cwd),
    batch_size=125,
    shuffle=True,
    target_size=(150, 150)
)


Found 12500 images belonging to 2 classes.
Found 12500 images belonging to 2 classes.


# 변형데이터

In [0]:
train_x=np.load(file='/content/drive/My Drive/train_data_150_img.npy',allow_pickle=True)
train_y=np.load(file='/content/drive/My Drive/train_data_150_label.npy',allow_pickle=True)

In [0]:
#width_shift_range=0.3,height_shift_range=0.3,horizontal_flip=True,vertical_flip=True,rotation_range=45,zoom_range=0.2
gen_arg=tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.3,height_shift_range=0.3)
arg_train=gen_arg.flow(x=train_x,y=train_y,batch_size=12500,shuffle=True)

# techer_student

In [0]:
vgg_model_ts=tf.keras.applications.VGG16(include_top=False,weights='imagenet',input_tensor=tf.keras.layers.Input(shape=(150,150,3)))


In [21]:
%%time
img_train,label_train=generator_2.next()
bottleneck_train=vgg_model_2.predict(img_train)

CPU times: user 1h 30min 1s, sys: 16 s, total: 1h 30min 17s
Wall time: 23min 18s


In [0]:
np.save(file='bottleneck_train_label',arr=label_train)
np.save(file='bottleneck_train',arr=bottleneck_train)

In [23]:
%%time
img_val,label_val=generator_2_test.next()
bottleneck_val=vgg_model_2.predict(img_val)

CPU times: user 1h 30min 3s, sys: 16.9 s, total: 1h 30min 20s
Wall time: 23min 20s


In [0]:
np.save(file='bottleneck_val_label',arr=label_val)
np.save(file='bottleneck_val',arr=bottleneck_val)

In [0]:
%cp bottleneck_val_label.npy /content/drive/My\ Drive
%cp bottleneck_val.npy /content/drive/My\ Drive
%cp bottleneck_train_label.npy /content/drive/My\ Drive
%cp bottleneck_train.npy /content/drive/My\ Drive


# 병목만들기


In [19]:
vgg_model_2=tf.keras.applications.VGG16(include_top=False,weights='imagenet',input_tensor=tf.keras.layers.Input(shape=(150,150,3)))


58892288/58889256 [==============================] - 1s 0us/step


In [26]:
#변형없음
%%time
img,label=generator_2.next()
bottleneck_test=vgg_model_2.predict(img)

CPU times: user 1h 29min 45s, sys: 17.8 s, total: 1h 30min 3s
Wall time: 23min 14s


In [0]:
#변형있음
%%time
arg_img,arg_label=arg_train.ne 

xt()
bottleneck_arg_train=vgg_model_2.predict(arg_img)

(512, 2)

# 모델정의

In [0]:
model_bt = tf.keras.Sequential()
model_bt.add(tf.keras.layers.Flatten())
model_bt.add(tf.keras.layers.Dense(256, activation='relu'))
model_bt.add(tf.keras.layers.Dropout(0.5))
model_bt.add(tf.keras.layers.Dense(2, activation='softmax'))

model_bt.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
model_bt.fit(bottleneck_train, label_train,
          epochs=5,
          batch_size=512,validation_data=(bottleneck_val,label_val)
          )

Train on 12500 samples, validate on 12500 samples
Epoch 1/5
12500/12500 [==============================] - 2s 150us/sample - loss: 0.1414 - accuracy: 0.9467 - val_loss: 0.2129 - val_accuracy: 0.9115
Epoch 2/5
12500/12500 [==============================] - 2s 152us/sample - loss: 0.1357 - accuracy: 0.9494 - val_loss: 0.2157 - val_accuracy: 0.9121
Epoch 3/5
12500/12500 [==============================] - 2s 150us/sample - loss: 0.1294 - accuracy: 0.9511 - val_loss: 0.2221 - val_accuracy: 0.9123
Epoch 4/5
12500/12500 [==============================] - 2s 147us/sample - loss: 0.1179 - accuracy: 0.9554 - val_loss: 0.2167 - val_accuracy: 0.9114
Epoch 5/5
12500/12500 [==============================] - 2s 148us/sample - loss: 0.1050 - accuracy: 0.9615 - val_loss: 0.2171 - val_accuracy: 0.9127


In [0]:
model_bt.save_weights('/content/save_weight')

# 평가

In [0]:
vgg_model_2=tf.keras.applications.VGG16(include_top=False,weights='imagenet',input_tensor=tf.keras.layers.Input(shape=(150,150,3)))

existing_model = tf.keras.Sequential()
existing_model.add(tf.keras.layers.Flatten())
existing_model.add(tf.keras.layers.Dense(256, activation='relu'))
existing_model.add(tf.keras.layers.Dropout(0.5))
existing_model.add(tf.keras.layers.Dense(2, activation='softmax'))

existing_model.build(vgg_model_2.output_shape)

In [0]:

existing_model(vgg_model_2.output)

exvgg_model=tf.keras.Model(inputs=vgg_model_2.input,outputs=existing_model.output)



In [16]:
exvgg_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

# 실험중

In [0]:
vgg_model_for_test=tf.keras.applications.VGG16(include_top=False,weights='imagenet',input_tensor=tf.keras.layers.Input(shape=(150,150,3)))

In [3]:
vgg_model_for_test.output_shape

(None, 4, 4, 512)

In [0]:

inputs_=tf.keras.layers.Input(shape=vgg_model_for_test.output_shape[1:])
x= tf.keras.layers.Flatten()(inputs_)
x= tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs_= tf.keras.layers.Dense(2, activation='softmax')(x)
existing_model2=tf.keras.Model(inputs=inputs_,outputs=outputs_)


In [35]:
existing_model2.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 4, 4, 512)]       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
Total params: 2,097,922
Trainable params: 2,097,922
Non-trainable params: 0
_________________________________________________________________


In [6]:
vgg_model_for_test.output

<tf.Tensor 'block5_pool/Identity:0' shape=(None, 4, 4, 512) dtype=float32>

In [0]:
inputs=tf.keras.layers.Input(shape=(150,150,3))
model1=tf.keras.applications.VGG16(include_top=False,weights='imagenet',input_tensor=inputs)
model2=existing_model2(model1.output)
exvgg_model2=tf.keras.Model(inputs=model1.input,outputs=model2)

In [0]:
exvgg_model2.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
existing_model2.load_weights('/content/save_weight')

In [48]:
exvgg_model2.evaluate(generator_2_test)

  ...
    to  
  ['...']
  5/100 [>.............................] - ETA: 21:57 - loss: 0.7607 - accuracy: 0.5440

KeyboardInterrupt: ignored

In [0]:
tf.keras.utils.plot_model(exvgg_model2, 'modle1_test.png', show_shapes=True)

# layer 잠그기

In [0]:
for layer in vgg_model_2.layers :
  layer.trainable=False

In [33]:
existing_model.load_weights('/content/save_weight')

In [0]:
exvgg_model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
exvgg_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [0]:
test_img,test_label=generator_2_test.next()

In [0]:
batch_image = np.expand_dims(test_img[1], 0)

In [40]:
exvgg_model.predict(batch_image)

array([[0.05833877, 0.9416612 ]], dtype=float32)

In [16]:
exvgg_model.evaluate(generator_2_test)

NameError: ignored

# 모델로드

In [0]:
exvgg_model.save('/content/5_6.h5')

In [0]:
%cp 5_6.h5 /content/drive/My Drive

In [0]:
from tensorflow.keras.models import load_model

model2 = load_model('/content/drive/My Drive/5_6.h5')


In [15]:
model2.predict(batch_image)

array([[0.9456012 , 0.05439877]], dtype=float32)